In [ ]:
%matplotlib inline

# a lot of this is hack and pasted from jup_plot_logs
# but prefer to hack it here, than fill jup_plot_logs with random specific stuff

import matplotlib.pyplot as plt
import math, glob, json, os, subprocess, time, datetime, sys, importlib
from os import path
from os.path import join
from collections import defaultdict, OrderedDict
import numpy as np
sys.path.append('..')

import ulfs
from ulfs.params import Params
from ulfs import graphing, graphing_common, graphing_indexes
importlib.reload(ulfs.graphing)
importlib.reload(ulfs.graphing_common)
importlib.reload(ulfs.graphing_indexes)

log_dir = '../logs'

import glob
from collections import defaultdict, OrderedDict

script = 'e2e_fixpoint'
refs = ['mll310']
# refs = ['mll186b', 'mll187', 'mll188', 'mll232', 'mll233']
value_keys = ['e2e_loss', 'e2e_acc', 'send_acc', 'recv_acc']
# filters = ['RNN', '3x10']
# filters = ['RNNHierarchical', '5x10']
filters = []
# max_step = 25000
max_step = 25000
units = ''
num_rows = 2
# units = 'thousands'

meta_by_grammar_by_meanings_by_link = defaultdict(dict)
last_refmodellinkmeanings = ''
import matplotlib.pyplot as plt
meta_by_grammar = {}
grammars = []
grammars_set = set()
meta_by_grammar_by_rmlm = OrderedDict()

for ref in refs:
    e2e_logfiles = glob.glob(f'{log_dir}/log_{script}_{ref}_*.log')
    for i, log_filepath in enumerate(e2e_logfiles):
    #     print(file)
    #     log_filepath = join(log_dir, file)
        num_lines = graphing_common.get_num_lines(log_filepath)
        if num_lines <= 3:
    #         print('skipping', file)
            continue
        meta = graphing_common.read_meta(log_filepath)
        meta['num_lines'] = num_lines
        argv = meta['argv']
        params = Params(meta['params'])
        _ref = params.ref
    #     print('ref', ref)
        if _ref != ref:
            continue
    #     print('ref', ref)
        meta['log_filepath'] = log_filepath
        meanings = f'{params.num_meaning_types}x{params.meanings_per_type}'
        grammar = params.grammar
        if params.corruptions is not None and params.corruptions != '':
            grammar = params.corruptions
        if grammar not in grammars_set:
            grammars.append(grammar)
            grammars_set.add(grammar)
        rmlm = f'{params.model}_{params.link}_{meanings}'
        exclude = False
        for filter in filters:
            if filter not in rmlm:
                exclude = True
        if exclude:
            continue
        # check for timeouts
        sup_lines = graphing_common.head(log_filepath, 3).split('\n')[1:3]
    #     print('sup_lines', sup_lines)
        for line in sup_lines:
            d = json.loads(line)
    #         print(d['terminate_reason'])
            if d['terminate_reason'] == 'timeout':
                exclude = True
                print('excluding for timeout', rmlm, grammar)
        if exclude:
            continue
        if rmlm not in meta_by_grammar_by_rmlm:
            meta_by_grammar_by_rmlm[rmlm] = {}
        meta_by_grammar_by_rmlm[rmlm][grammar] = meta

for rmlm, meta_by_grammar in meta_by_grammar_by_rmlm.items():
    print(rmlm)
    for grammar, meta in meta_by_grammar.items():
        print('    ', grammar, meta['log_filepath'])

for rmlm, meta_by_grammar in meta_by_grammar_by_rmlm.items():
    plt.figure(figsize=(20, 3.5 * num_rows * num_rows))
    plt.cla()
    for j, value_key in enumerate(value_keys):
        plt.subplot(num_rows, 4 // num_rows, j + 1)
        for grammar in grammars:
            if grammar not in meta_by_grammar:
#                 print(f'warning {grammar} not found')
                continue
            meta = meta_by_grammar[grammar]
            log_filepath = meta['log_filepath']
            graphing.plot_logfile2(
                log_filepath, step_key='episode', skip_record_types=['sup_train_res'], value_key=value_key,
                label=grammar, title=value_key, max_step=max_step, units=units
            )
    print('')
    print(' '.join(rmlm.split('_')[1:]))
    plt.show()
